In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

# Load and preprocess the data
dataset = np.loadtxt("/kaggle/input/train-test-amazon1/train_test_amazon1.sample", delimiter=' ', dtype= int)
# Split features and labels
X = dataset[:,2:-1]
y = dataset[:,-1:]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data to have three dimensions (height, width, and channels)
X_train_image = X_train.reshape((-1, X_train.shape[1], 1))
X_test_image = X_test.reshape((-1, X_test.shape[1], 1))

# Determine batch size
batch_size = 64

# Create the input layer
inputs = Input(shape=(X_train_image.shape[1], X_train_image.shape[2], 1))

# Inception-like module
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[2], (5, 5), padding='same', activation='relu')(x)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_conv = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(maxpool)

    inception_block = concatenate([conv1x1, conv3x3, conv5x5, maxpool_conv], axis=-1)
    return inception_block

# Build the Inception model
x = inception_module(inputs, filters=[64, 128, 128, 64])
x = inception_module(x, filters=[128, 192, 192, 128])
x = inception_module(x, filters=[192, 256, 256, 192])

# Global Average Pooling
x = GlobalAveragePooling2D()(x)

# Fully Connected Layer
output = Dense(1, activation='sigmoid', name='output')(x)  # Assuming you have 4 classes

# Create the model
model = Model(inputs, output)
optimizer = Adam(learning_rate=0.0001)
# Compile the model with an appropriate optimizer, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    X_train_image,
    y_train,
    epochs=30,
    batch_size=batch_size,
    validation_data=(X_test_image, y_test)
)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score, confusion_matrix, f1_score
import matplotlib.pyplot as plt

model.save('Inception_without_SMOTE.h5')
scores = model.evaluate(X_test_image, y_test)
np.savetxt('scores_without_SMOTE.txt', scores, delimiter=',', fmt='%f')
print("scores:",scores)
y_pred_probs = model.predict(X_test_image)
np.savetxt('y_pred_probs_without_SMOTE.txt', y_pred_probs, delimiter=',', fmt='%f')
print("y_pred_probs:",y_pred_probs)
y_pred = np.round(y_pred_probs)
np.savetxt('y_pred_without_SMOTE.txt', y_pred, delimiter=',', fmt='%f')
print("y_pred:", y_pred)
fpr, tpr, _ = roc_curve(y_test.ravel(), y_pred_probs.ravel())
print("fpr:",fpr)
np.savetxt('fpr_without_SMOTE.txt', fpr, delimiter=',', fmt='%f')
print("tpr:",tpr)
np.savetxt('tpr_without_SMOTE.txt', tpr, delimiter=',', fmt='%f')
roc_auc = roc_auc_score(y_test.ravel(), y_pred_probs.ravel())
print("roc_auc:",roc_auc)
precision, recall, _ = precision_recall_curve(y_test.ravel(), y_pred_probs.ravel())
print("precision:",precision)
print("recall:", recall)
np.savetxt('precision_without_SMOTE.txt', precision, delimiter=',', fmt='%f')
np.savetxt('recall_without_SMOTE.txt', recall, delimiter=',', fmt='%f')
average_precision = average_precision_score(y_test.ravel(), y_pred_probs.ravel())

cm = confusion_matrix(y_test.ravel(), y_pred.ravel())
f1 = f1_score(y_test.ravel(), y_pred.ravel())
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("ROC AUC:", roc_auc)
print("Precision-Recall AUC:", average_precision)
print("Confusion Matrix:")
#print(cm)
print("F1-Score:", f1)
print("True Positive Rate (TPR):", tpr_value)
print("False Positive Rate (FPR):", fpr_value)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('/kaggle/working/roc_curve_real_without_SMOTE.png')
plt.show()

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % average_precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('/kaggle/working/precision_recall_curve_real_without_SMOTE.png')
plt.show()

with open('/kaggle/working/results_inception_real.txt', 'w') as file:
    file.write(f"ROC AUC: {roc_auc}\n")
    file.write(f"Precision-Recall AUC: {average_precision}\n")
    file.write(f"Confusion Matrix:\n{cm}\n")
    file.write(f"F1-Score: {f1}\n")
    file.write(f"True Positive Rate (TPR): {tpr_value}\n")
    file.write(f"False Positive Rate (FPR): {fpr_value}\n")
    file.write(f"tpr: {tpr}\n")
    file.write(f"fpr: {fpr}\n")
    file.write(f"precision: {precision}\n")
    file.write(f"recall: {recall}\n")
    file.write(f"y_pred: {y_pred}\n")

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(average_precision))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision.tolist()
recall_list = recall.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": float(f1), 
    #"Average Precision": precision, 
    #"Average Recall": float(recall), 
    "Average True Positive Rate": float(tpr_value), 
    "Average False Positive Rate": float(fpr_value), 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": average_precision,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/Inception - Real.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)